In [ ]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-me7ocbbw
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-me7ocbbw
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [ ]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <stdint.h> // uint32_t

using element_t = uint32_t;

In [ ]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [ ]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <random>  // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {

/**
 * Generates and returns a vector of random uniform data of a given length, n,
 * for any integral type. Input range will be [0, 2n].
 */
template < typename T >
std::vector< T > generate_uniform( std::size_t n )
{
    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20240916;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 2 * n);

    std::vector< T > random_data( n ); // init array
    std::generate( std::begin( random_data )
                 , std::end  ( random_data )
                 , [ &rng, &distrib ](){ return static_cast< T >( distrib( rng ) ); });

    return random_data;
}

} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "algorithm_choices.h"
#pragma once

#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {
namespace cpu {

void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace cpu


namespace gpu {

void run_gpu_soln( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "cpu_baseline.cu"
/**
 * CPU methods that the GPU should outperform.
 */

#include "algorithm_choices.h"

#include <algorithm> // std::sort()
#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

namespace csc485b {
namespace a1      {
namespace cpu     {

/**
 * Simple solution that just sorts the whole array with a built-in sort
 * function and then resorts the last portion in the opposing order with
 * a second call to that same built-in sort function.
 */
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    std::sort( data, data + num_elements, std::less< element_t >{} );
    std::sort( data + invert_at_pos, data + num_elements, std::greater< element_t >{} );
}

/**
 * Run the single-threaded CPU baseline that students are supposed to outperform
 * in order to obtain higher grades on this assignment. Times the execution and
 * prints to the standard output (e.g., the screen) that "wall time." Note that
 * the functions takes the input by value so as to not perturb the original data
 * in place.
 */
void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n )
{
    auto const cpu_start = std::chrono::high_resolution_clock::now();
    opposing_sort( data.data(), switch_at, n );
    auto const cpu_end = std::chrono::high_resolution_clock::now();

    std::cout << "CPU Baseline time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(cpu_end - cpu_start).count()
              << " ns" << std::endl;

    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
}

} // namespace cpu
} // namespace a1
} // namespace csc485b

In [144]:
%%cuda_group_save -g "source" -n "gpu_solution.cu"
/**
 * The file in which you will implement your GPU solutions!
 */

#include "algorithm_choices.h"

#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

#include "cuda_common.h"

namespace csc485b {
namespace a1      {
namespace gpu     {

/**
 * The CPU baseline benefits from warm caches because the data was generated on
 * the CPU. Run the data through the GPU once with some arbitrary logic to
 * ensure that the GPU cache is warm too and the comparison is more fair.
 */
__global__
void warm_the_gpu( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    // We know this will never be true, because of the data generator logic,
    // but I doubt that the compiler will figure it out. Thus every element
    // should be read, but none of them should be modified.
    if( th_id < num_elements && data[ th_id ] > num_elements * 100 )
    {
        ++data[ th_id ]; // should not be possible.
    }
}

/**
 * Your solution. Should match the CPU output.
 */

__global__
void opposing_sort(element_t * data, std::size_t invert_at_pos, std::size_t num_elements)
{
    // Calculate the next power of 2 for the given input size
    std::size_t padded_size = 1;
    while (padded_size < num_elements) {
        padded_size <<= 1;
    }

    // Step 1: Allocate shared memory with padding
    __shared__ element_t shared_data[1024];

    // Initialize padded value. They will be large values so as to finalize at the end of the sorted array
    element_t pad_value = 1e9;

    // Compute the global thread ID and local thread ID within the block
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;
    int const local_th_id = threadIdx.x;

    // Exit thread if it is not needed (ie there are fewer inputs than the possible number of threads)
    if (local_th_id >= padded_size) return;

    // Step 2: Load data from global memory to shared memory
    if (th_id < padded_size) {
        if (th_id < num_elements) {
            shared_data[local_th_id] = data[th_id]; // Load actual data
        } else {
            shared_data[local_th_id] = pad_value; // Load padding value
        }
    }

    // Ensure all threads in the block have loaded their data into shared memory
    __syncthreads();

    // Step 3: Perform bitonic sort using shared memory
    // Only use the appropriate number of threads for the given array size
    if (th_id < padded_size) {
        // For-loop concerning each stage of the bitonic sort. log(n) stages for input of size n, where n is a power of 2
        for (int k = 2; k <= padded_size; k <<= 1) {
            // For-loop concerning each step of each stage. Every stage has an equivalent number of steps. i.e. stage 1 has 1 step, stage 2 has 2 steps...
            for (int j = k >> 1; j > 0; j >>= 1) {

                // Get the index of the current swap partner
                int i_swap = local_th_id ^ j;

                // If the th_id is less than the swap partner
                if (local_th_id < i_swap && i_swap < blockDim.x) {
                    // If we are NOT on the last stage
                    if ((k != padded_size) || ((k == padded_size) && (padded_size != num_elements))) {
                        // Swap the values in ascending order
                        if ((local_th_id & k) == 0) {
                            if (shared_data[local_th_id] > shared_data[i_swap]) {
                                element_t t = shared_data[local_th_id];
                                shared_data[local_th_id] = shared_data[i_swap];
                                shared_data[i_swap] = t;
                            }
                        // Swap the values in descending order
                        } else {
                            if (shared_data[local_th_id] < shared_data[i_swap]) {
                                element_t t = shared_data[local_th_id];
                                shared_data[local_th_id] = shared_data[i_swap];
                                shared_data[i_swap] = t;
                            }
                        }
                    }
                    // Handle the last step where we apply the inversion at the specified position ONLY for inputs where the size is exactly a power of 2
                    else if (padded_size == num_elements) {
                        if (local_th_id < invert_at_pos) {
                            if (shared_data[local_th_id] > shared_data[i_swap]) {
                                element_t t = shared_data[local_th_id];
                                shared_data[local_th_id] = shared_data[i_swap];
                                shared_data[i_swap] = t;
                            }
                        } else if (local_th_id >= invert_at_pos) {
                            if (shared_data[local_th_id] < shared_data[i_swap]) {
                                element_t t = shared_data[local_th_id];
                                shared_data[local_th_id] = shared_data[i_swap];
                                shared_data[i_swap] = t;
                            }
                        }
                    }
                }

                // Synchronize threads within the block before the next stage
                __syncthreads();
            }
        }
    }
    // Step 3.5
    // If the input size is not a power of 2, then perform normal bitonic sort on the array, and reverse elements after 3n/4
    if (padded_size > num_elements) {
        // Only use the threads between invert_at_pos and num_elements
        if ((local_th_id >= invert_at_pos) && (local_th_id < num_elements)) {
            // Math equation to get the appropriate swap partners
            int swap_partner = num_elements - (local_th_id - invert_at_pos) - 1;
            // Swap the elements using the threads
            if (local_th_id < swap_partner) {
                if (shared_data[local_th_id] < shared_data[swap_partner]) {
                    element_t t = shared_data[local_th_id];
                    shared_data[local_th_id] = shared_data[swap_partner];
                    shared_data[swap_partner] = t;
                }
            }
        }
    }
    __syncthreads();

    // Step 4: Copy sorted data back to global memory, excluding padding
    if (th_id < num_elements) {
        data[th_id] = shared_data[local_th_id];
    }
}



/**
 * Performs all the logic of allocating device vectors and copying host/input
 * vectors to the device. Times the opposing_sort() kernel with wall time,
 * but excludes set up and tear down costs such as mallocs, frees, and memcpies.
 */
void run_gpu_soln( std::vector< element_t > data, std::size_t switch_at, std::size_t n )
{
    // Kernel launch configurations. Feel free to change these.
    // This is set to maximise the size of a thread block on a T4, but it hasn't
    // been tuned. It's not known if this is optimal.
    std::size_t const threads_per_block = 1024;
    std::size_t const num_blocks =  ( n + threads_per_block - 1 ) / threads_per_block;

    // Allocate arrays on the device/GPU
    element_t * d_data;
    cudaMalloc( (void**) & d_data, sizeof( element_t ) * n );
    CHECK_ERROR("Allocating input array on device");

    // Copy the input from the host to the device/GPU
    cudaMemcpy( d_data, data.data(), sizeof( element_t ) * n, cudaMemcpyHostToDevice );
    CHECK_ERROR("Copying input array to device");

    // Warm the cache on the GPU for a more fair comparison
    warm_the_gpu<<< num_blocks, threads_per_block>>>( d_data, switch_at, n );

    // Time the execution of the kernel that you implemented
    auto const kernel_start = std::chrono::high_resolution_clock::now();
    opposing_sort<<< num_blocks, threads_per_block>>>( d_data, switch_at, n );
    auto const kernel_end = std::chrono::high_resolution_clock::now();
    CHECK_ERROR("Executing kernel on device");

    // After the timer ends, copy the result back, free the device vector,
    // and echo out the timings and the results.
    cudaMemcpy( data.data(), d_data, sizeof( element_t ) * n, cudaMemcpyDeviceToHost );
    CHECK_ERROR("Transferring result back to host");
    cudaFree( d_data );
    CHECK_ERROR("Freeing device memory");

    std::cout << "GPU Solution time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(kernel_end - kernel_start).count()
              << " ns" << std::endl;

    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
}

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [149]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <cstddef>  // std::size_t type
#include <iostream> // std::cout, std::endl
#include <vector>

#include "algorithm_choices.h"
#include "data_generator.h"
#include "data_types.h"
#include "cuda_common.h"

int main()
{
    std::size_t const n = 576;
    std::size_t const switch_at = 3 * ( n >> 2 ) ;

    auto data = csc485b::a1::generate_uniform< element_t >( n );
    /*
    for (auto &d:data) {
        std::cout << d << ' ';
    }
    */
    std::cout << '\n';
    csc485b::a1::cpu::run_cpu_baseline( data, switch_at, n );
    csc485b::a1::gpu::run_gpu_soln( data, switch_at, n );

    return EXIT_SUCCESS;
}

In [150]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"


CPU Baseline time: 27391 ns
1 3 7 11 12 14 17 22 22 23 23 27 27 30 30 30 33 36 38 38 40 40 47 47 48 51 56 58 58 62 63 65 65 66 67 67 69 74 79 79 81 84 84 86 87 92 95 100 101 104 111 112 113 115 116 117 119 119 119 124 130 133 134 134 136 137 142 143 144 145 146 146 146 146 149 154 154 155 155 160 161 163 166 167 169 171 174 177 177 183 183 185 187 188 188 192 194 194 194 197 199 201 209 210 214 216 217 222 222 232 234 236 239 241 242 242 246 247 249 249 249 257 261 261 263 263 265 265 265 266 270 270 272 273 273 275 280 281 284 286 288 288 289 305 306 308 309 311 312 314 314 314 315 317 318 321 323 323 324 326 329 332 332 334 335 337 342 342 342 345 346 347 349 349 350 360 362 368 372 379 381 382 382 382 384 387 387 390 391 392 393 393 394 397 399 400 401 402 404 410 410 410 414 417 418 423 424 426 426 427 428 429 431 432 438 441 442 444 445 445 446 452 453 453 455 456 458 459 459 463 465 466 467 469 470 475 479 482 482 486 486 490 490 496 497 505 505 507 509 510 519 520 522 524 528 5